# Mixture of Gaussian Experiments

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# utility func

In [2]:
def save_plot(x, epoch=None):
    # make directory if there is not
    path = "gaussian"
    if not os.path.isdir(path):
        os.makedirs(path)    
    
    plt.figure(figsize=(5,5))
    plt.scatter(x[:,0],x[:,1],s=2, color=matplotlib.colors.cnames["orangered"])
    plt.ylim(-4.0, 4.0)
    plt.xlim(-4.0, 4.0)
    plt.legend()
    plt.savefig(os.path.join(path, "gaussian" + str(epoch) + ".png"))
    plt.close()

In [3]:
import seaborn as sns
def kdeplot(x, epoch=None):
    # make directory if there is not
    path = "kdeplot"
    if not os.path.isdir(path):
        os.makedirs(path)    
    ax = plt.figure()
    ax = sns.kdeplot(x[:,0], x[:,1], shade=True, cmap="Greens",n_levels=20, clip=[[-2, 2]]*2)
    bg_color = sns.color_palette("Greens", n_colors=256)[0]
    ax.set_axis_bgcolor(bg_color)
    ax.savefig(os.path.join(path, "gaussian" + str(epoch) + ".png"))

In [4]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [5]:
# plot images
def save_imgs(images, plot_dim=(5,12), size=(12,5), epoch=None):
    # make directory if there is not
    path = "generated_figures"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 60
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((96, 96, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

In [6]:
def fc(x, output_shape, name):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    initializer = tf.random_normal_initializer(0., 0.001)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        bias = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
        x = tf.nn.bias_add(x, bias)

    return x

In [7]:
import math
def gaussian_mixture_circle(batch_size, num_cluster=8, scale=2, std=0.2):
    rand_indices = np.random.randint(0, num_cluster, size=batch_size)
    base_angle = math.pi * 2 / num_cluster
    angle = rand_indices * base_angle - math.pi / 2
    mean = np.zeros((batch_size, 2), dtype=np.float32)
    mean[:, 0] = np.cos(angle) * scale
    mean[:, 1] = np.sin(angle) * scale
    return np.random.normal(mean, std**2, (batch_size, 2)).astype(np.float32)

# model

In [8]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.z_dim = 256
        self.h_dim = 128
        self.o_dim = 2
        self.initializer = tf.contrib.layers.xavier_initializer()

    def __call__(self, x, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            with tf.variable_scope("fc1"):
                weight = tf.get_variable("W", shape=[self.z_dim, self.h_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.h_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias
                #x = tf.nn.relu(x)
                x = tf.nn.tanh(x)

            with tf.variable_scope("fc2"):
                weight = tf.get_variable("W", shape=[self.h_dim, self.o_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.o_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return x

In [9]:
class Discriminator:
    def __init__(self, batch_size=64):
        self.reuse = False
#        self.initializer = tf.truncated_normal(stddev=0.8)
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 2
        self.h_dim = 128
        self.o_dim = 1
        
    def __call__(self, x, training=False, name=''):
        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            x = x * 0.25

            with tf.variable_scope("fc1"):
                weight = tf.get_variable("W", shape=[self.X_dim, self.h_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.h_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias
                x = tf.nn.tanh(x)

            with tf.variable_scope("fc2"):
                weight = tf.get_variable("W", shape=[self.h_dim, self.o_dim], initializer=self.initializer)
                bias = tf.get_variable("b", shape=[self.o_dim], initializer=self.initializer)
                x = tf.matmul(x, weight) + bias
#                x = tf.nn.tanh(x)
                
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return x

In [10]:
class GAN:
    def __init__(self, batch_size=64):
        self.batch_size = batch_size
        self.g = Generator()
        self.d = Discriminator()
        self.z_dim = 256
        self.batch_size = batch_size
        self.epoch_save_gauss = np.array([0, 1, 5, 10, 20, 50, 100])*1000
        self.epoch_save_metrics = 5000
        self.losses = {"dis_loss":[], "gen_loss":[]}
        self.gaussians = []

    def loss(self, y, y_gen):
        
        d_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y, labels=tf.ones_like(y))) + tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_gen, labels=tf.zeros_like(y_gen))) # 怪しい
        g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_gen, labels=tf.ones_like(y_gen)))    # 怪しい
        #d_loss = -tf.reduce_mean(tf.log(y) + tf.log(1. - y_gen)) 
        #g_loss = tf.reduce_mean(tf.log(1-y_gen))
        return g_loss, d_loss

    def train(self, batch_size=64, epochs=100001):
        z = tf.placeholder(tf.float32, shape=[None, self.z_dim])
        x = tf.placeholder(tf.float32, shape=[None, 2])
        x_gen = self.g(z)
        y = self.d(x)
        y_gen = self.d(x_gen)
        g_loss, d_loss = self.loss(y, y_gen)
        
        #t_vars = tf.global_variables()
        #g_vars = [var for var in t_vars if "g" in var.name]
        #d_vars = [var for var in t_vars if "d" in var.name]

        opt_d = tf.train.AdamOptimizer(1e-4, beta1=0.5).minimize(d_loss, var_list=self.d.variables)
        opt_g = tf.train.AdamOptimizer(1e-4, beta1=0.5).minimize(g_loss, var_list=self.g.variables)

        z_fix = np.random.uniform(-1, 1, size=[1024, self.z_dim])
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            steps = 10
            for step in range(steps):
                # 潜在変数と生成データ
                z_gen = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim])
                # 訓練データを生成
                X_tr = gaussian_mixture_circle(self.batch_size)
                _, gen_loss = self.sess.run([opt_g, g_loss], feed_dict={z: z_gen, x: X_tr})             
                _, dis_loss = self.sess.run([opt_d, d_loss], feed_dict={z: z_gen, x: X_tr})
                if step == 0:
                    dis_var = self.sess.run(self.d.variables)
                
            np_dis_var = np.array(dis_var)
            for (dst, src) in zip(self.d.variables, np_dis_var):
                dst = src

            self.sess.run(self.d.variables)
            del np_dis_var

            self.losses["dis_loss"].append(dis_loss)
            self.losses["gen_loss"].append(gen_loss)
            
            if epoch in self.epoch_save_gauss:

                x_ = self.sess.run(x_gen, feed_dict={z: z_fix})
                self.gaussians.append(x_)
                print(epoch)

            if epoch % self.epoch_save_metrics == 0:
                save_metrics(self.losses, epoch)
            
        x_tr = gaussian_mixture_circle(1024)
        self.gaussians.append(x_tr)

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

    def save_gauss_dist(self):
        i = 0
        for gauss in self.gaussians:
            i+=1
            save_plot(gauss, str(i))

    def save_gauss_dists(self):
        path = "gaussian_unrolled"
        if not os.path.isdir(path):
            os.makedirs(path)

        fig = plt.figure(figsize=(15, 2))
        epochs = self.epoch_save_gauss = np.array([0, 1, 5, 10, 20, 50, 100, "target"])
        i = 0
        print(len(self.gaussians))
        for x in self.gaussians:

            ax = fig.add_subplot(1, len(self.gaussians), i+1)
            ax.scatter(x[:,0],x[:,1],s=2, color=matplotlib.colors.cnames["darkorange"], alpha=0.3)

            if i==0:
                ax.set_ylabel("vanilla GAN")
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_ylim(-4.0, 4.0)
            ax.set_xlim(-4.0, 4.0)   
            print(i)
            if i < 7:
                ax.patch.set_facecolor(matplotlib.colors.cnames["whitesmoke"])
                ax.set_title('epoch: ' + str(epochs[i]) + "k")
            else:
                ax.patch.set_facecolor(matplotlib.colors.cnames["lightgrey"])   
                ax.set_title('target')
            i += 1

        plt.axis("off")
        plt.tight_layout()   
        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.09, hspace=0.07)
        plt.axvline(x=-4.0)

        plt.savefig(os.path.join(path, "gaussian_dist.png"))
        plt.close()

In [11]:
if __name__ == "__main__":
    gan = GAN()
    gan.train()
    gan.save_gauss_dist()
    gan.save_gauss_dists()
#    gan.g()

0


/home/ujitoko/anaconda3/lib/python3.6/site-packages/matplotlib/transforms.py:661: RuntimeWarning: invalid value encountered in absolute
  inside = ((abs(dx0 + dx1) + abs(dy0 + dy1)) == 0)


1000
5000
10000
20000


KeyboardInterrupt: 